In [ ]:
import requests
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Layer, LSTM, Dense, Flatten
from keras_self_attention import SeqSelfAttention
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol=VSAT&apikey=ZP6OGKNI4IBLBCKF'
r = requests.get(url)
data = r.json()

In [ ]:
df = pd.DataFrame.from_dict(dict(data['Weekly Adjusted Time Series']), orient='index', dtype=float)
df = df.sort_index(ascending=True)

features = ['5. adjusted close']
df_features = df[features]

target = '5. adjusted close'

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df_features)

def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length - 1):
        x = data[i:(i + seq_length)]
        y = data[i + seq_length, df_features.columns.get_loc(target)]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

seq_length = 10
X, y = create_sequences(scaled_data, seq_length)

train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2]))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2]))

model = tf.keras.Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(SeqSelfAttention(attention_activation='relu'))
model.add(Flatten())
model.add(tf.keras.layers.Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=100, batch_size=32)

y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_pred), len(features)-1)), y_pred), axis=1))[:, -1]
y_test_inv = scaler.inverse_transform(np.concatenate((np.zeros((len(y_test), len(features)-1)), y_test.reshape(-1,1)), axis=1))[:, -1]


#rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
#print(f'RMSE: {rmse}')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.0884
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0042
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0034
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 0.0033
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0036
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0029
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0034
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.0034
Epoch 9/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0033
Epoch 10/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0031
Epoch 11/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0033
Epoch 12/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0034
Epoch 13/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0033
Epoch 14/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0037
Epoch 15/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.0032
Epo

In [ ]:
last_sequence = scaled_data[-seq_length:].reshape(1, seq_length, 1)
next_week_scaled = model.predict(last_sequence)

predicted_price = scaler.inverse_transform(
    np.concatenate((np.zeros((1, len(features) - 1)), next_week_scaled), axis=1)
)[0, -1]

last_actual_price = df_features.iloc[-1][target]

percentage_change = ((predicted_price - last_actual_price) / last_actual_price) * 100

print(f'Last actual stock price: {last_actual_price}')
print(f'Predicted stock price for next week: {predicted_price}')
print(f'Expected percentage change: {percentage_change:.2f}%')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step
Last actual stock price: 11.5
Predicted stock price for next week: 10.9327492415905
Expected percentage change: -4.93%


In [ ]:
#url = 'https://api.polygon.io/v2/aggs/ticker/AAPL/range/1/day/2023-01-24/2025-01-24?adjusted=true&sort=asc&apiKey=vVw1vPhIehpCgHJJjudROTZwCn2dkU57'
#r = requests.get(url)
#data = r.json()

In [ ]:
import plotly.graph_objects as go

scale = 1/scaler.scale_[0]
fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index[train_size + seq_length + 1: ], y=y_test.flatten()*scale, mode='lines', name='Actual'))

fig.add_trace(go.Scatter(x=df.index[train_size + seq_length + 1: ], y=y_pred.flatten()*scale, mode='lines', name='Predicted'))

fig.update_layout(title='Viasat Stock Price Prediction',
                  xaxis_title='Date',
                  yaxis_title='Closing Price',
                  xaxis_rangeslider_visible=True)


fig.show()